In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.style.use('default')
from sklearn import cluster
from sklearn.cluster import KMeans
import math
import plotly.graph_objects as pgo
import plotly
plotly.offline.init_notebook_mode(connected=True)
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD
import numpy as np
from collections import Counter

In [ ]:
user_ratings = pd.read_csv('user_ratings.csv')
user_history = pd.read_csv('user_history.csv')
ratings_table = user_ratings.pivot(index = 'USER ID', columns = 'PRODUCT', values = 'RATING')
products_list = list(ratings_table.columns)

In [ ]:
user_history = pd.read_csv('user_history.csv')
user_history_vectors = user_history.drop(columns=['USER ID']).to_numpy()

In [ ]:
k = 3
kmeans = cluster.KMeans(k)
kmeans.fit(user_history_vectors)
pd.Series(kmeans.labels_).value_counts()

In [204]:
def fit_batchsize_one(lr,epochs,p_u,product_ratings):
    q = np.random.normal(size=(100,))
    
    for e in range(epochs):
        gr_q = np.zeros(100)
        
        for o in range(len(product_ratings)):
            
            r = product_ratings[o]
            p = p_u[o]
            gr_q += find_gradient_sigmoid(r, p, q)
        q = q-lr*gr_q/len(product_ratings)
        if (e+1)%1000==0:
            print('Epoch {}/{} - training set total rating error: {}'.format(e+1, epochs, find_training_loss(p_u,q,product_ratings)))
    return q

def find_training_loss(p_u,q,product_ratings):
    s=0
    for i in range(len(product_ratings)):
        r = product_ratings[i]
        p = p_u[i]
        sig = sigmoid(np.dot(p,q))
        s+=(10*sig-r)**2
    return s
    
def sigmoid(x):
    return 1/(1+math.exp(-x))
    
def find_gradient_sigmoid(r, p, q):
    pu_T_qm = np.dot(p,q)
    sig = sigmoid(pu_T_qm)
    gr_q = 2*(10*sig-r)*10*sig*(1-sig)*p    
    return gr_q

def get_ratings(prod, clust_num, labels, user_history,ratings_table):
    curr_clust_labels = pd.Series(labels == clust_num, index=pd.Index(user_history['USER ID']))

    M = user_history.set_index('USER ID')[curr_clust_labels].to_numpy()
    meen = np.mean(M,axis=0,keepdims=True)
    curr_clust_users_hist = user_history.set_index(user_history['USER ID']).drop(columns=['USER ID'])[curr_clust_labels]

    curr_clust_raters = curr_clust_labels.loc[list(ratings_table.index)]

    prod_ratings=ratings_table[curr_clust_raters][prod].dropna()
    prod_raters_vects = curr_clust_users_hist.loc[prod_ratings.index].to_numpy()
    prod_raters_vects = prod_raters_vects - meen
    lr = 0.008
    epochs = 6000
    
    q = fit_batchsize_one(lr,epochs,prod_raters_vects,list(prod_ratings))
    
    ratings = (curr_clust_users_hist.to_numpy()-meen)@q
    
    return pd.Series([round(10*sigmoid(ratings[i])) for i in range(len(ratings))],index=curr_clust_users_hist.index).rename('RATING')

In [206]:
df = pd.DataFrame()
for clust_num in [2,1,0]:
    for product in products_list:
        print('cluster:',clust_num,' product:',product)
        r = ratings_table[product]
        curr_clust_raters = pd.Series(labels==clust_num,index=user_history['USER ID']).loc[r.index]
        r = r[curr_clust_raters].dropna()
        x = get_ratings(product, clust_num, labels, user_history, ratings_table) #get predictions
        x.loc[r.index] = r #correct the predictions for known ratings
        f = pd.DataFrame(x) #convert the predicted ratings into a dataframe
        f.reset_index(level=0, inplace=True) #make USER ID into a column and reset the index
        f['PRODUCT'] = product #introduct a column for product name
        df = pd.concat([df,f],ignore_index=True) #append this dataframe onto results produced this far

cluster: 2  product: alaska basil
Epoch 1000/6000 - training set total rating error: 498.9303368980026
Epoch 2000/6000 - training set total rating error: 48.23743658509222
Epoch 3000/6000 - training set total rating error: 35.24839049098423
Epoch 4000/6000 - training set total rating error: 34.68637884488725
Epoch 5000/6000 - training set total rating error: 34.654782652397245
Epoch 6000/6000 - training set total rating error: 34.6528219182719
cluster: 2  product: alcohol future
Epoch 1000/6000 - training set total rating error: 1340.1160152624168
Epoch 2000/6000 - training set total rating error: 361.73689770647877
Epoch 3000/6000 - training set total rating error: 57.48325731477381
Epoch 4000/6000 - training set total rating error: 33.96597532497734
Epoch 5000/6000 - training set total rating error: 31.924637203134285
Epoch 6000/6000 - training set total rating error: 31.705994405732522
cluster: 2  product: alice ticket
Epoch 1000/6000 - training set total rating error: 987.906992793

Epoch 2000/6000 - training set total rating error: 47.035204689207404
Epoch 3000/6000 - training set total rating error: 35.97382867079714
Epoch 4000/6000 - training set total rating error: 35.44066039638267
Epoch 5000/6000 - training set total rating error: 35.4070176932325
Epoch 6000/6000 - training set total rating error: 35.404736547778846
cluster: 2  product: diana chariot
Epoch 1000/6000 - training set total rating error: 1388.989639257891
Epoch 2000/6000 - training set total rating error: 215.13575158305932
Epoch 3000/6000 - training set total rating error: 43.95107414089549
Epoch 4000/6000 - training set total rating error: 35.62039760181607
Epoch 5000/6000 - training set total rating error: 34.97153473300444
Epoch 6000/6000 - training set total rating error: 34.90813145765877
cluster: 2  product: diesel jason
Epoch 1000/6000 - training set total rating error: 1122.4904988772496
Epoch 2000/6000 - training set total rating error: 123.78467962571408
Epoch 3000/6000 - training set

Epoch 4000/6000 - training set total rating error: 107.35008647274208
Epoch 5000/6000 - training set total rating error: 54.25429795683033
Epoch 6000/6000 - training set total rating error: 38.653519998838235
cluster: 2  product: judge sunday
Epoch 1000/6000 - training set total rating error: 1507.9833515996472
Epoch 2000/6000 - training set total rating error: 70.5056934475732
Epoch 3000/6000 - training set total rating error: 37.66472327573511
Epoch 4000/6000 - training set total rating error: 36.7360608386235
Epoch 5000/6000 - training set total rating error: 36.69643731555638
Epoch 6000/6000 - training set total rating error: 36.69443370583145
cluster: 2  product: july miami
Epoch 1000/6000 - training set total rating error: 756.5770222088455
Epoch 2000/6000 - training set total rating error: 85.28659923527259
Epoch 3000/6000 - training set total rating error: 37.61089964565941
Epoch 4000/6000 - training set total rating error: 34.20460678555061
Epoch 5000/6000 - training set total

Epoch 6000/6000 - training set total rating error: 34.69033790483087
cluster: 2  product: nurse october
Epoch 1000/6000 - training set total rating error: 1168.5019185895285
Epoch 2000/6000 - training set total rating error: 209.19949727898722
Epoch 3000/6000 - training set total rating error: 50.402503217401964
Epoch 4000/6000 - training set total rating error: 36.877664100689856
Epoch 5000/6000 - training set total rating error: 35.088785646187006
Epoch 6000/6000 - training set total rating error: 34.76315519812306
cluster: 2  product: office triton
Epoch 1000/6000 - training set total rating error: 1035.624386503595
Epoch 2000/6000 - training set total rating error: 441.3578382183265
Epoch 3000/6000 - training set total rating error: 124.63192862714148
Epoch 4000/6000 - training set total rating error: 52.15484906334319
Epoch 5000/6000 - training set total rating error: 38.986833250076714
Epoch 6000/6000 - training set total rating error: 35.914418279493155
cluster: 2  product: pard

Epoch 1000/6000 - training set total rating error: 1404.094807155667
Epoch 2000/6000 - training set total rating error: 316.37703576902186
Epoch 3000/6000 - training set total rating error: 66.17213898577866
Epoch 4000/6000 - training set total rating error: 38.916688263736454
Epoch 5000/6000 - training set total rating error: 35.546970916076134
Epoch 6000/6000 - training set total rating error: 35.07701942257529
cluster: 2  product: sheriff java
Epoch 1000/6000 - training set total rating error: 337.62965331634535
Epoch 2000/6000 - training set total rating error: 36.0663418407829
Epoch 3000/6000 - training set total rating error: 32.58115178212052
Epoch 4000/6000 - training set total rating error: 32.47848885427996
Epoch 5000/6000 - training set total rating error: 32.47482982668526
Epoch 6000/6000 - training set total rating error: 32.474690751314384
cluster: 2  product: silk gizmo
Epoch 1000/6000 - training set total rating error: 502.1303692369367
Epoch 2000/6000 - training set to

...

Epoch 3000/6000 - training set total rating error: 1398.6356982681048
Epoch 4000/6000 - training set total rating error: 1398.4389384856584
Epoch 5000/6000 - training set total rating error: 1398.4300516613132
Epoch 6000/6000 - training set total rating error: 1398.4296291702615
cluster: 0  product: diesel jason
Epoch 1000/6000 - training set total rating error: 4820.326330950273
Epoch 2000/6000 - training set total rating error: 4203.857148510988
Epoch 3000/6000 - training set total rating error: 4159.9471498704925
Epoch 4000/6000 - training set total rating error: 4158.241231314178
Epoch 5000/6000 - training set total rating error: 4158.153273627746
Epoch 6000/6000 - training set total rating error: 4158.147734861071
cluster: 0  product: diploma vocal
Epoch 1000/6000 - training set total rating error: 4516.62956601022
Epoch 2000/6000 - training set total rating error: 4098.742778521261
Epoch 3000/6000 - training set total rating error: 4091.1132388796873
Epoch 4000/6000 - training se

Epoch 5000/6000 - training set total rating error: 3803.9942733637604
Epoch 6000/6000 - training set total rating error: 3803.983790937581
cluster: 0  product: july miami
Epoch 1000/6000 - training set total rating error: 3688.215190861082
Epoch 2000/6000 - training set total rating error: 3364.6672684134983
Epoch 3000/6000 - training set total rating error: 3355.3948077016375
Epoch 4000/6000 - training set total rating error: 3354.974029218041
Epoch 5000/6000 - training set total rating error: 3354.949149148086
Epoch 6000/6000 - training set total rating error: 3354.947438827865
cluster: 0  product: kermit desert
Epoch 1000/6000 - training set total rating error: 2666.7776025511816
Epoch 2000/6000 - training set total rating error: 2410.298845181778
Epoch 3000/6000 - training set total rating error: 2407.6931550770983
Epoch 4000/6000 - training set total rating error: 2407.619401001558
Epoch 5000/6000 - training set total rating error: 2407.6165083598275
Epoch 6000/6000 - training set

Epoch 1000/6000 - training set total rating error: 3028.6259699086145
Epoch 2000/6000 - training set total rating error: 2410.3027545545465
Epoch 3000/6000 - training set total rating error: 2402.3703443751333
Epoch 4000/6000 - training set total rating error: 2402.2216587277408
Epoch 5000/6000 - training set total rating error: 2402.2171055395997
Epoch 6000/6000 - training set total rating error: 2402.2169169103486
cluster: 0  product: pardon thomas
Epoch 1000/6000 - training set total rating error: 2069.3759594573826
Epoch 2000/6000 - training set total rating error: 1499.887640576354
Epoch 3000/6000 - training set total rating error: 1495.511767153989
Epoch 4000/6000 - training set total rating error: 1495.4330021277783
Epoch 5000/6000 - training set total rating error: 1495.430927813879
Epoch 6000/6000 - training set total rating error: 1495.4308687406767
cluster: 0  product: parody athena
Epoch 1000/6000 - training set total rating error: 6572.952824100459
Epoch 2000/6000 - traini

Epoch 3000/6000 - training set total rating error: 4708.161835496337
Epoch 4000/6000 - training set total rating error: 4707.67400166954
Epoch 5000/6000 - training set total rating error: 4707.6391554460015
Epoch 6000/6000 - training set total rating error: 4707.6361738267815
cluster: 0  product: silk gizmo
Epoch 1000/6000 - training set total rating error: 5713.466562654119
Epoch 2000/6000 - training set total rating error: 5044.824632787268
Epoch 3000/6000 - training set total rating error: 5020.237676147902
Epoch 4000/6000 - training set total rating error: 5019.14634754223
Epoch 5000/6000 - training set total rating error: 5019.0716856897925
Epoch 6000/6000 - training set total rating error: 5019.065103721942
cluster: 0  product: silver plate
Epoch 1000/6000 - training set total rating error: 3232.8667023481903
Epoch 2000/6000 - training set total rating error: 2853.281266174237
Epoch 3000/6000 - training set total rating error: 2848.577717616002
Epoch 4000/6000 - training set tota

Epoch 5000/6000 - training set total rating error: 3078.2597247167164
Epoch 6000/6000 - training set total rating error: 3078.2594462744205
cluster: 0  product: turtle pierre
Epoch 1000/6000 - training set total rating error: 4268.50667770932
Epoch 2000/6000 - training set total rating error: 4118.072206931088
Epoch 3000/6000 - training set total rating error: 4115.907488440203
Epoch 4000/6000 - training set total rating error: 4115.839545574185
Epoch 5000/6000 - training set total rating error: 4115.836340768361
Epoch 6000/6000 - training set total rating error: 4115.836134466886
cluster: 0  product: update vibrate
Epoch 1000/6000 - training set total rating error: 3064.241040990299
Epoch 2000/6000 - training set total rating error: 2898.1386399485996
Epoch 3000/6000 - training set total rating error: 2896.408087161525
Epoch 4000/6000 - training set total rating error: 2896.371176648645
Epoch 5000/6000 - training set total rating error: 2896.3700555423843
Epoch 6000/6000 - training se

In [207]:
df = df[['USER ID','PRODUCT','RATING']]
df.to_csv('predictions.csv')

In [218]:
# This could have been done in the loop above where for every iteration of the outer loop, we save the dataframe for the corresponding cluster
# Alas, I thought of this only after I started the long loop to generate predictions

df1 = df.loc[0:149999]
df2 = df.loc[150000:299999]
df3 = df.loc[300000:449999]

In [230]:
# save predictions for each cluster

df1.to_csv('predictions_cluster_a.csv')
df2.to_csv('predictions_cluster_b.csv')
df3.to_csv('predictions_cluster_c.csv')